# Requirements

In [12]:
import matplotlib.pyplot as plt
import random
%matplotlib inline

# Implementation

First we define a class to evolve the cellular autotomaton.  A runner is created for a specific rule, specified by a number between 0 and 255.  For instance, rule 47 would translate to:
* $000 \mapsto 1$
* $001 \mapsto 1$
* $010 \mapsto 1$
* $011 \mapsto 1$
* $100 \mapsto 0$
* $101 \mapsto 1$
* $110 \mapsto 0$
* $111 \mapsto 0$

In [2]:
class AutomatonRunner:
    
    def __init__(self, rule_nr):
        self._compute_rules(rule_nr)
        
    def _compute_rules(self, rule_nr):
        self._rules = []
        for _ in range(8):
            self._rules.append(rule_nr % 2)
            rule_nr //= 2
    
    def _apply_rule(self, environment):
        env_nr = 4*environment[0] + 2*environment[1] + environment[2]
        return self._rules[env_nr]
    
    def _make_environment(self, automaton, i):
        if 0 < i < len(automaton) - 1:
            return automaton[i - 1:i + 2]
        elif i == 0:
            return [automaton[-1]] + automaton[:2]
        elif i == len(automaton) - 1:
            return automaton[i - 1:] + [automaton[0]]
        
    def next_generation(self, automaton):
        ng_automaton = []
        for i in range(len(automaton)):
            environment = self._make_environment(automaton, i)
            ng_automaton.append(self._apply_rule(environment))
        return ng_automaton
                                
    def evolve(self, automaton, nr_generations):
        generations = [automaton]
        for _ in range(nr_generations):
            generations.append(self.next_generation(generations[-1]))
        return generations
    
    def __str__(self):
        auto_str = ''
        for i, result in enumerate(self._rules):
            auto_str += f'{i//4 % 2}{i//2 % 2}{i % 2} -> {result}\n'
        return auto_str

# Iterator class

Typcally, we want to run a simulation for a large number of cellular automata. Since the number of automata with $N$ cells is $2^N$, it is unfeasible to do this for even moderate values of $N$.  Hence we will define a factory that generates a given number of automata of a certain length.  The factory will be an iterator, so that one can loop over the instances.

In [7]:
class RandomAutomatonFactory:
    
    _max_nr_automata: int
    _nr_automaton: int
    _nr_cells: int
    
    def __init__(self, nr_cells, nr_automata):
        self._max_nr_automata = nr_automata
        self._nr_cells = nr_cells
        
    def __iter__(self):
        self._nr_automaton = 0
        return self
        
    def __next__(self):
        if self._nr_automaton < self._max_nr_automata:
            self._nr_automaton += 1
            return random.choices((0, 1), k=self._nr_cells)
        else:
            raise StopIteration()

In Python, a class implements an iterator when it has to define the `__iter__` and `__next__` methods.  The former initializes the iteration object, the latter returns the next instance in the iterator.  If `__next__` should not return any more items, i.e., when the iteration is done, it raises a `StopIterator` exception.

In [8]:
for automaton in RandomAutomatonFactory(nr_cells=10, nr_automata=5):
    print(automaton)

[1, 1, 0, 0, 1, 0, 1, 1, 0, 0]
[0, 1, 1, 1, 0, 1, 1, 1, 1, 0]
[0, 0, 0, 1, 1, 1, 1, 1, 1, 0]
[1, 0, 1, 1, 1, 1, 1, 0, 1, 1]
[1, 1, 1, 1, 0, 0, 0, 0, 0, 1]


Note that you can view `RandomAutomatonFactory` as a factory class, as the name already suggests.  So this is an illustration of combining (even mixing) two design patters, factory and iterator.

# Generator function

This can also be achieved by creating a generator.

In [9]:
def random_automata(nr_cells, nr_automata):
    for _ in range(nr_automata):
        yield random.choices((0, 1), k=nr_cells)

In [10]:
for automaton in random_automata(nr_cells=10, nr_automata=5):
    print(automaton)

[0, 1, 0, 1, 1, 1, 1, 1, 1, 1]
[1, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 0, 1, 1, 1, 0, 0, 0]
[0, 0, 0, 1, 1, 1, 1, 0, 0, 0]
[1, 1, 0, 1, 0, 0, 0, 0, 1, 1]


# Application

Now we can run a number of randomly generated automata with a given rule.

In [14]:
nr_cols = 10
nr_automata = 100
nr_generations = 80
figure, axes = plt.subplots(nr_automata//nr_cols, nr_cols, figsize=(100, 100))
rule_nr = 47
runner = AutomatonRunner(rule_nr)
nr_cells = 20
for automatron_nr, automaton in enumerate(random_automata(nr_cells=nr_cells, nr_automata=nr_automata)):
    row_nr = automatron_nr//nr_cols
    col_nr = automatron_nr % nr_cols
    generations = runner.evolve(automaton, nr_generations)
    axes[row_nr][col_nr].imshow(generations, aspect='auto')
    axes[row_nr][col_nr].set_title(str(rule_nr));
plt.tight_layout()